# SANDAG Sector Estimates Analysis
This is a calculator that takes in EDD data and aggregates it to specified SANDAG sectors. This also takes in estimates data and aggregates those to SANDAG sectors as well. Then a comparison between the two is drawn. 

See Purva's Excel for more information: https://sandag.sharepoint.com/:x:/r/qaqc/_layouts/15/Doc.aspx?sourcedoc=%7BCFD80313-6A49-43E6-B776-50C85CD17DD4%7D&file=EDD_Forecast%20Output%20Industry%20Level%20Jobs%20Comparison_QA.xlsx&action=default&mobileredirect=true&cid=9c4c556c-a447-4a46-8e56-ab5f1c95311a

In [10]:
import pandas as pd
import urllib.request  # For downloading the xlsx file
import pandas as pd
from sodapy import Socrata
import ssl
import sqlalchemy

# Grabbing EDD Data
See documentation here under the 'Employment Controls' section for more information on where the data comes from: https://sandag.sharepoint.com/:w:/r/qaqc/_layouts/15/doc2.aspx?sourcedoc=%7BE29C6BBB-F51B-43D8-87C8-2AF2CB34D5DA%7D&file=MGRABased_input_ABM_2019_process_notes%20-%20Copy.docx&action=default&mobileredirect=true&isSPOFile=1&clickparams=eyJBcHBOYW1lIjoiVGVhbXMtRGVza3RvcCIsIkFwcFZlcnNpb24iOiIyNy8yMjA3MDMwMDgxNSIsIkhhc0ZlZGVyYXRlZFVzZXIiOmZhbHNlfQ%3D%3D&cid=5a7ad9e6-56cc-41b4-b447-1bc8ccf0eff6

Documentation stipulates that data is taken from here: https://data.edd.ca.gov/Industry-Information-/Current-Employment-Statistics-CES-Annual-Average-1/pwx8-ztk5/data

In [11]:
# Sector Level Data
client = Socrata("data.edd.ca.gov", None)
results = client.get_all("pwx8-ztk5", area_name='San Diego-Carlsbad MSA', year=2019)
results_df = pd.DataFrame.from_records(results)

In [12]:
# Grab the EDD sectors that we are interested in 
edd_breakdown = ['Mining and Logging','Total Farm','Construction','Manufacturing','Wholesale Trade','Retail Trade','Utilities','Transportation and Warehousing','Information','Finance and Insurance','Real Estate and Rental and Leasing','Professional, Scientific and Technical S','Management of Companies and Enterprises','Administrative and Support and Waste Ser','Educational Services','Health Care and Social Assistance','Arts, Entertainment, and Recreation','Accommodation','Food Services and Drinking Places','Other Services','Federal Government excluding Department of Defense','Department of Defense','State Government Education','State Government Excluding Education','Local Government Education','Local Government Excluding Education']

subset_data = results_df[results_df['industry_title'].isin(edd_breakdown)]
subset_data = subset_data[['industry_title', 'current_employment']]
edd_data = subset_data.set_index('industry_title').T

In [13]:
edd_data

industry_title,Total Farm,Manufacturing,Construction,Mining and Logging,Finance and Insurance,Information,Transportation and Warehousing,Utilities,Retail Trade,Wholesale Trade,...,Other Services,Food Services and Drinking Places,Accommodation,"Arts, Entertainment, and Recreation",Health Care and Social Assistance,Educational Services,Administrative and Support and Waste Ser,Management of Companies and Enterprises,"Professional, Scientific and Technical S",Real Estate and Rental and Leasing
current_employment,9700,115700,84000,400,46400,23500,29700,4600,145600,44000,...,56400,139400,32100,30200,186400,30200,87000,23800,145000,30200


# Grabbing Estimates Data

In [14]:
#estimates_data = pd.read_csv('C:/Users/cra/OneDrive - San Diego Association of Governments/SANDAG-Sector-Estimates-Analysis/region_ind_DS42.csv')
#Adjust the year
#estimates_data = estimates_data[estimates_data['year'] == 2020].reset_index()

In [15]:
# estimates_data = pd.read_csv('C:/Users/cra/OneDrive - San Diego Association of Governments/SANDAG-Sector-Estimates-Analysis/mgra15_region_ind_QA.csv')
estimates_data = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/data/MGRA13 Updated Data/mgra13_update_region_ind_QA.csv')

# Creating Final Output

In [16]:
# Setting up the final dataframe
final_df = pd.DataFrame()

# Setting up the proper code titles
final_df.index = ['Mining, logging, total farm', 'Arts, Entertainment & Recreation', 'Construction', 'Department of Defense', 'Federal Government excluding Department of Defense', 'Health Care and Social Assistance', 'Accomodation', 'Manufacturing', 'Other Services', 'Professional Services', 'Government Education', 'Educational Services', 'Food Services', 'Retail Trade', 'Government Non-education', 'Transportation & Warehousing', 'Utilities', 'Wholesale Trade']

# Originally Set the Data to Zero
final_df['Estimates Data'] = 0
final_df['EDD Data'] = 0

# Filling in the final dataframe with  the correct values 

In [17]:
# Mining, logging, total farm 
final_df['Estimates Data']['Mining, logging, total farm'] = int(estimates_data['emp_ag'])
final_df['EDD Data']['Mining, logging, total farm'] = int(edd_data['Mining and Logging']) + int(edd_data['Total Farm'])

# Arts, Entertainment & Recreation
final_df['Estimates Data']['Arts, Entertainment & Recreation'] = int(estimates_data['emp_amusement'])
final_df['EDD Data']['Arts, Entertainment & Recreation'] = int(edd_data['Arts, Entertainment, and Recreation'])

# Construction
final_df['Estimates Data']['Construction'] = int(estimates_data['emp_const_bldg_office']) + int(estimates_data['emp_const_bldg_prod']) + int(estimates_data['emp_const_non_bldg_office']) + int(estimates_data['emp_const_non_bldg_prod'])
final_df['EDD Data']['Construction'] = int(edd_data['Construction'])

# Department of Defence 
final_df['Estimates Data']['Department of Defense'] = int(estimates_data['emp_fed_mil'])
final_df['EDD Data']['Department of Defense'] = int(edd_data['Department of Defense']) # I don't see a number 27


# Federal Government excluding Department of Defense
final_df['Estimates Data']['Federal Government excluding Department of Defense'] = int(estimates_data['emp_fed_non_mil'])
final_df['EDD Data']['Federal Government excluding Department of Defense'] = int(edd_data['Federal Government excluding Department of Defense'])

# Health Care & Social Assistance
final_df['Estimates Data']['Health Care and Social Assistance'] = int(estimates_data['emp_health'])
final_df['EDD Data']['Health Care and Social Assistance'] = int(edd_data['Health Care and Social Assistance'])

# Accommodation
final_df['Estimates Data']['Accomodation'] = int(estimates_data['emp_hotel'])
final_df['EDD Data']['Accomodation'] = int(edd_data['Accommodation'])


# Manufacturing
final_df['Estimates Data']['Manufacturing'] = int(estimates_data['emp_mfg_office']) + int(estimates_data['emp_mfg_prod'])
final_df['EDD Data']['Manufacturing'] = int(edd_data['Manufacturing']) 

# Other Services
final_df['Estimates Data']['Other Services'] = int(estimates_data['emp_personal_svcs_office']) + int(estimates_data['emp_personal_svcs_retail'])
final_df['EDD Data']['Other Services'] = int(edd_data['Other Services'])

# Professional Services 
final_df['Estimates Data']['Professional Services'] = int(estimates_data['emp_prof_bus_svcs']) + int(estimates_data['emp_prof_bus_svcs_bldg_maint'])
final_df['EDD Data']['Professional Services'] = int(edd_data['Information']) + int(edd_data['Finance and Insurance']) + int(edd_data['Real Estate and Rental and Leasing']) + int(edd_data['Professional, Scientific and Technical S']) + int(edd_data['Management of Companies and Enterprises']) + int(edd_data['Administrative and Support and Waste Ser'])

# Government Education - Spelling
final_df['Estimates Data']['Government Education'] = int(estimates_data['emp_public_ed'])
final_df['EDD Data']['Government Education'] = int(edd_data['State Government Education']) + int(edd_data['Local Government Education'])

# Educational Services
final_df['Estimates Data']['Educational Services'] = int(estimates_data['emp_pvt_ed_k12']) + int(estimates_data['emp_pvt_ed_post_k12_oth'])
final_df['EDD Data']['Educational Services'] = int(edd_data['Educational Services'])


# Food Services
final_df['Estimates Data']['Food Services'] = int(estimates_data['emp_restaurant_bar'])
final_df['EDD Data']['Food Services'] = int(edd_data['Food Services and Drinking Places']) #This is an assumption, not explicitly stated

# Retail Trade 
final_df['Estimates Data']['Retail Trade'] = int(estimates_data['emp_retail'])
final_df['EDD Data']['Retail Trade'] = int(edd_data['Retail Trade'])

# Government Non-eduction
final_df['Estimates Data']['Government Non-education'] = int(estimates_data['emp_state_local_gov_blue']) + int(estimates_data['emp_state_local_gov_ent']) + int(estimates_data['emp_state_local_gov_white'])
final_df['EDD Data']['Government Non-education'] = int(edd_data['State Government Excluding Education']) + int(edd_data['Local Government Excluding Education'])

# Transportation and Warehousing
final_df['Estimates Data']['Transportation & Warehousing'] = int(estimates_data['emp_trans'])
final_df['EDD Data']['Transportation & Warehousing'] = int(edd_data['Transportation and Warehousing'])

# Utilities
final_df['Estimates Data']['Utilities'] = int(estimates_data['emp_utilities_office']) + int(estimates_data['emp_utilities_prod'])
final_df['EDD Data']['Utilities'] = int(edd_data['Utilities'])

# Wholesale Trade
final_df['Estimates Data']['Wholesale Trade'] = int(estimates_data['emp_whsle_whs'])
final_df['EDD Data']['Wholesale Trade'] = int(edd_data['Wholesale Trade'])


final_df['Diff'] = final_df['Estimates Data'] - final_df['EDD Data']

In [18]:
final_df.loc['Sum'] = [sum(final_df['Estimates Data']), sum(final_df['EDD Data']), (sum(final_df['Estimates Data']) - sum(final_df['EDD Data']))]

In [19]:
final_df

,Estimates Data,EDD Data,Diff
"Mining, logging, total farm",9905,10100,-195
"Arts, Entertainment & Recreation",49907,30200,19707
Construction,92078,84000,8078
Department of Defense,129474,23700,105774
Federal Government excluding Department of Defense,11454,23900,-12446
Health Care and Social Assistance,215961,186400,29561
Accomodation,33604,32100,1504
Manufacturing,115687,115700,-13
Other Services,70566,56400,14166
Professional Services,398762,355900,42862


In [22]:
# final_df.to_excel('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-58 2019 Base Year Forecast Output QC/Output/mgra13_EDD_SANDAG_Sector_Diff.xlsx')